In [7]:
%config IPCompleter.greedy=True
%matplotlib inline

import csv, json, os, sys
import requests
from time import sleep

import obspy
from obspy import read

from obspy.core.event import read_events
from obspy import read_inventory

## Download and Sample
the following:
* net=XB
* sta=ELYSE
* cha=BHU,BHV,BHW
* loc=02

In [14]:
def download_data_availability(data_path='../data/mars/'):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass
    
    payload = {
        'option': 'com_ajax',
        'data': 'ELYSE',
        'format': 'json',
        'module': 'seis_data_available'
    }
    r = requests.post('https://www.seis-insight.eu/en/science/seis-data/seis-data-availability', payload)
    with open(os.path.join(data_path, 'data_availability.json'), 'wb') as f:
        f.write(r.content)

In [4]:
download_data_availability()

In [5]:
with open(os.path.join('../data/mars/', 'data_availability.json'), 'r') as f:
    raw_ava = json.load(f)

In [6]:
ava = []
for t in raw_ava['data']:
    if t['network'] == 'XB' and t['location'] == '00' and t['channel'] == 'BHU':
        ava.append(t)

In [23]:
ava[0]

IndexError: list index out of range

example data req: `http://ws.ipgp.fr/fdsnws/dataselect/1/query?network=XB&amp;station=ELYSE&amp;startTime=2019-02-12T02:43:01&amp;endTime=2019-02-12T04:14:38&amp;location=02&amp;channel=BH?&amp;nodata=404`

In [50]:
def download_mseed(event, channel='BH?', data_path='../data/mars/raw'):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass

    payload = {
        'network': event['network'],
        'station': event['station'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'location': event['location'],
        'channel': channel
    }
    
    req = requests.get('http://ws.ipgp.fr/fdsnws/dataselect/1/query', params=payload)
    file_name = '-'.join([event['network'], event['station'], event['location'], event['startTime'], event['endTime']]) + '.mseed'
    print('downloading: %s' % file_name)
    path = os.path.join(data_path, file_name)
    with open(path, 'wb') as c:
        c.write(req.content)
    return path

In [9]:
# download all examples:
for event in ava:
    download_mseed(event)
    sleep(1)

downloading: XB-ELYSE-02-2019-02-12T02:43:01.313000-2019-02-12T04:14:37.259000.mseed
downloading: XB-ELYSE-02-2019-02-12T06:39:54.299000-2019-02-12T07:10:02.247000.mseed
downloading: XB-ELYSE-02-2019-02-15T08:14:08.907000-2019-02-15T08:42:48.854000.mseed


KeyboardInterrupt: 

In [10]:
with open('../data/mars/raw/catelog.json', 'w') as f:
    json.dump(ava, f)

In [8]:
file_names = list(filter(lambda f: os.path.splitext(f)[1] == '.mseed', os.listdir('../data/mars/raw')))
print('num of files: %i' % len(file_names))

num of files: 145


In [9]:
stream = obspy.read(os.path.join('../data/mars/raw', file_names[1]))

In [10]:
stream.filter('highpass', freq=0.1)
stream.resample(sampling_rate=20.0, window='hanning', no_filter=True, strict_length=False)

15 Trace(s) in Stream:
XB.ELYSE.02.BHU | 2019-05-01T18:59:49.095000Z - 2019-05-02T01:00:23.045000Z | 20.0 Hz, 432680 samples
XB.ELYSE.02.BHU | 2019-05-02T00:59:31.054000Z - 2019-05-02T07:10:38.004000Z | 20.0 Hz, 445340 samples
XB.ELYSE.02.BHU | 2019-05-02T07:09:46.034000Z - 2019-05-02T08:00:13.984000Z | 20.0 Hz, 60560 samples
XB.ELYSE.02.BHU | 2019-05-02T07:59:22.031000Z - 2019-05-02T14:00:13.981000Z | 20.0 Hz, 433040 samples
XB.ELYSE.02.BHU | 2019-05-02T13:59:21.992000Z - 2019-05-02T19:00:13.942000Z | 20.0 Hz, 361040 samples
XB.ELYSE.02.BHV | 2019-05-01T18:59:49.095000Z - 2019-05-02T01:00:23.045000Z | 20.0 Hz, 432680 samples
XB.ELYSE.02.BHV | 2019-05-02T00:59:31.054000Z - 2019-05-02T07:10:38.004000Z | 20.0 Hz, 445340 samples
XB.ELYSE.02.BHV | 2019-05-02T07:09:46.034000Z - 2019-05-02T08:00:13.984000Z | 20.0 Hz, 60560 samples
XB.ELYSE.02.BHV | 2019-05-02T07:59:22.031000Z - 2019-05-02T14:00:13.981000Z | 20.0 Hz, 433040 samples
XB.ELYSE.02.BHV | 2019-05-02T13:59:21.992000Z - 2019-05-02T19

In [11]:
stream[0].meta

         network: XB
         station: ELYSE
        location: 02
         channel: BHU
       starttime: 2019-05-01T18:59:49.095000Z
         endtime: 2019-05-02T01:00:23.045000Z
   sampling_rate: 20.0
           delta: 0.05
            npts: 432680
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'R', 'number_of_records': 1707, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 8547328})
      processing: ["ObsPy 1.2.1: filter(options={'freq': 0.1}::type='highpass')", "ObsPy 1.2.1: resample(no_filter=True::sampling_rate=20.0::strict_length=False::window='hanning')"]

## Visualize data around recorded marsquakes

In [33]:
event_times = [
    ('2019-12-20T13:12:36', '2019-12-20T13:27:56', '2.4Hz-B') # 2.4Hz -- B
]

non_event_times = [
    
]

In [3]:
event_example_dir = '../data/mars/event_examples'

In [34]:
for t in event_times:
    event = {
        'network': 'XB',
        'station': 'ELYSE',
        'startTime': t[0],
        'endTime': t[1],
        'location': '02'
    }
    download_mseed(event, channel='BH?', data_path=event_example_dir)
    sleep(1)

downloading: XB-ELYSE-02-2019-12-20T13:12:36-2019-12-20T13:27:56.mseed


In [8]:
event_file_names = list(filter(lambda f: os.path.splitext(f)[1] == '.mseed', os.listdir(event_example_dir)))

In [9]:
stream = obspy.read(os.path.join(event_example_dir, event_file_names[1]))
stream.filter('bandpass', freqmin=0.1, freqmax=9.0)

/home/b/anaconda3/envs/seis/lib/python3.7/site-packages/obspy/signal/headers.py:93: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ], align=True)


3 Trace(s) in Stream:
XB.ELYSE.02.BHU | 2019-12-20T13:12:24.187000Z - 2019-12-20T13:28:02.737000Z | 20.0 Hz, 18772 samples
XB.ELYSE.02.BHV | 2019-12-20T13:12:22.837000Z - 2019-12-20T13:28:14.737000Z | 20.0 Hz, 19039 samples
XB.ELYSE.02.BHW | 2019-12-20T13:12:20.337000Z - 2019-12-20T13:28:15.337000Z | 20.0 Hz, 19101 samples

In [37]:
events = read_events(os.path.join('data', 'events_extended_mars_preferredorigin_2020-01-01.xml'))

In [76]:
for pick in events[0].picks:
    print(pick.phase_hint + ':' + str(pick.time))

P_spectral_end:2019-12-31T17:46:12.696898Z
P_spectral_start:2019-12-31T17:33:53.449867Z
Peak_M2.4:2019-12-31T17:39:21.130288Z
noise_end:2019-12-31T17:29:31.305531Z
noise_start:2019-12-31T17:23:19.060572Z
end:2019-12-31T17:48:39.497726Z
start:2019-12-31T17:33:14.128216Z


In [69]:
events[10].picks

[Pick
	     resource_id: ResourceIdentifier(id="smi:insight.mqs/Pick/20200107105246.21803.66690")
	            time: UTCDateTime(2019, 12, 26, 15, 6, 45, 837740)
	     waveform_id: WaveformStreamID(network_code='XB', station_code='ELYSE', channel_code='BHN', location_code='02')
	       filter_id: ResourceIdentifier(id="smi:local/sc3filter/BW(6,2.2,2.8)__STALTA(100,1000)")
	      phase_hint: 'end'
	 evaluation_mode: 'manual'
	   creation_info: CreationInfo(agency_id='mqs', author='sc', creation_time=UTCDateTime(2020, 1, 7, 10, 52, 46, 218144)),
 Pick
	     resource_id: ResourceIdentifier(id="smi:insight.mqs/Pick/20191227170730.34993.231138")
	            time: UTCDateTime(2019, 12, 26, 14, 51, 10, 926410)
	     waveform_id: WaveformStreamID(network_code='XB', station_code='ELYSE', channel_code='BHZ', location_code='02')
	       filter_id: ResourceIdentifier(id="smi:local/sc3filter/BW_HP(3,0.001)")
	      phase_hint: 'start'
	 evaluation_mode: 'manual'
	   creation_info: CreationInfo(age

In [91]:
event_samples = []

for event in events:
    name = None
    
    for des in event.event_descriptions:
        if des.type == 'earthquake name':
            name = des.text
    
    for pick in event.picks:
        if pick.phase_hint == 'P_spectral_start':
            start = pick.time - 10
            end = pick.time + 10
            event_samples.append((name + '_P_spectral_start', start, end))
        elif pick.phase_hint == 'Peak_M2.4':
            start = pick.time - 10
            end = pick.time + 10
            event_samples.append((name + 'Peak_M2.4', start, end)) 
        

In [93]:
event_samples[-20:-1]

[('S0168aPeak_M2.4', 2019-05-18T18:11:22.650295Z, 2019-05-18T18:11:42.650295Z),
 ('S0167b_P_spectral_start',
  2019-05-17T19:30:00.624615Z,
  2019-05-17T19:30:20.624615Z),
 ('S0167a_P_spectral_start',
  2019-05-17T16:47:19.689413Z,
  2019-05-17T16:47:39.689413Z),
 ('S0154aPeak_M2.4', 2019-05-04T07:14:10.645246Z, 2019-05-04T07:14:30.645246Z),
 ('S0154a_P_spectral_start',
  2019-05-04T07:07:00.663595Z,
  2019-05-04T07:07:20.663595Z),
 ('S0152aPeak_M2.4', 2019-05-02T07:30:22.495265Z, 2019-05-02T07:30:42.495265Z),
 ('S0152a_P_spectral_start',
  2019-05-02T07:29:43.752239Z,
  2019-05-02T07:30:03.752239Z),
 ('S0151aPeak_M2.4', 2019-05-01T03:21:24.504456Z, 2019-05-01T03:21:44.504456Z),
 ('S0151a_P_spectral_start',
  2019-05-01T03:13:40.948606Z,
  2019-05-01T03:14:00.948606Z),
 ('S0133aPeak_M2.4', 2019-04-12T18:16:42.301108Z, 2019-04-12T18:17:02.301108Z),
 ('S0133a_P_spectral_start',
  2019-04-12T18:14:36.398722Z,
  2019-04-12T18:14:56.398722Z),
 ('S0132aPeak_M2.4', 2019-04-11T13:33:55.589827Z

In [94]:
for sample in event_samples[0:10]:
    event = {
        'network': 'XB',
        'station': 'ELYSE',
        'startTime': str(sample[1]),
        'endTime': str(sample[2]),
        'location': '02'
    }
    path = download_mseed(event, channel='BH?', data_path='..data/mars/20sec_events/')
    try:
        stream  = obspy.read(path)
    except: continue
    stream.filter('bandpass', freqmin=0.1, freqmax=9.0)
    stream.plot(outfile = os.path.join('../data/mars/20sec_event_images/', sample[0] + '.png'))
    sleep(1)

downloading: XB-ELYSE-02-2019-12-31T17:33:43.449867Z-2019-12-31T17:34:03.449867Z.mseed
downloading: XB-ELYSE-02-2019-12-31T17:39:11.130288Z-2019-12-31T17:39:31.130288Z.mseed
downloading: XB-ELYSE-02-2019-12-30T18:27:17.388199Z-2019-12-30T18:27:37.388199Z.mseed
downloading: XB-ELYSE-02-2019-12-30T18:32:58.696223Z-2019-12-30T18:33:18.696223Z.mseed
downloading: XB-ELYSE-02-2019-12-30T15:49:58.687286Z-2019-12-30T15:50:18.687286Z.mseed
downloading: XB-ELYSE-02-2019-12-30T15:52:22.352790Z-2019-12-30T15:52:42.352790Z.mseed
downloading: XB-ELYSE-02-2019-12-29T14:40:45.640024Z-2019-12-29T14:41:05.640024Z.mseed
downloading: XB-ELYSE-02-2019-12-29T14:42:01.755136Z-2019-12-29T14:42:21.755136Z.mseed
downloading: XB-ELYSE-02-2019-12-28T18:11:49.797885Z-2019-12-28T18:12:09.797885Z.mseed
downloading: XB-ELYSE-02-2019-12-28T18:04:58.257692Z-2019-12-28T18:05:18.257692Z.mseed


In [60]:
str(event_samples[0][1])

'2019-12-31T17:33:14.128216Z'